# Create Heat Map for significant Processing pre-mRNA genes

This notebook looks at the significant genes in at least one cancer in the DNA Replication pathway. Pancancer heat maps are created with circle size showing significance and color showing differences in median.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

import plot_utils as p

# Step 1: Run GSEA for significant genes in at least 1 cancer

First read in sig_pval_all_proteins.csv into a df. This csv file contains only genes with a significant p-value in at least one cancer. Then run GSEA using a list of genes from the df.

In [2]:
root = R'~\Github\WhenMutationsDontMatter\PTEN\Step_3_trans_effect\csv'
sig_df = pd.read_csv(root+R'\sig_pval_heatmap.csv')

prot_list = list(sig_df.Proteomics) # list of genes with a sig pval in >= 1 cancer
prot_enr = gp.enrichr(gene_list = prot_list, description='Tumor_partition', gene_sets='Reactome_2016', 
                       outdir='/Enrichr')

In [3]:
prot_enr.res2d.head()

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,Processing of Capped Intron-Containing Pre-mRN...,116/193,1.379053e-53,2.109952e-50,0,0,4.603878,560.363828,NUP107;NUP188;EIF4A3;HNRNPU;EFTUD2;SNRPD2;SNRP...,Reactome_2016
1,Gene Expression Homo sapiens R-HSA-74160,434/1631,1.734808e-53,1.327128e-50,0,0,2.038257,247.619892,TDRKH;RPL4;ATF2;MDC1;NUP107;HNRNPU;EHMT1;PHAX;...,Reactome_2016
2,mRNA Splicing - Major Pathway Homo sapiens R-H...,85/134,3.384052e-42,1.725866e-39,0,0,4.858892,463.973174,EIF4A3;HNRNPU;YBX1;PRPF19;USP39;ELAVL1;EFTUD2;...,Reactome_2016
3,mRNA Splicing Homo sapiens R-HSA-72172,85/144,1.081367e-38,4.136230e-36,0,0,4.521469,395.266853,EIF4A3;HNRNPU;YBX1;PRPF19;USP39;ELAVL1;EFTUD2;...,Reactome_2016
4,Infectious disease Homo sapiens R-HSA-5663205,123/348,8.726101e-27,2.670187e-24,0,0,2.707379,162.452135,RPL4;NUP107;NUP188;RPL10A;RPL9;PSMD8;RPS15;PSM...,Reactome_2016


# Step 2: Get the list of significant genes 

In [4]:
index = 0
trans = prot_enr.res2d.Genes[index]
genes = trans.split(';')
print(prot_enr.res2d.Term[index])
print('total genes:',len(genes))

Processing of Capped Intron-Containing Pre-mRNA Homo sapiens R-HSA-72203
total genes: 116


# Step 3: Create HeatMap

Slice out genes from the DNA Replication pathway from the df with genes sig in > 1 cancer.

In [5]:
# sig > 1 cancer
bool_df = sig_df.Proteomics.isin(genes)
plot_df = sig_df[bool_df]
len(plot_df.Proteomics.unique())

116

In [6]:
# Only include p-values < a certain cutoff
a = 0.01
plot_df = plot_df.loc[plot_df['P_Value'] <= a]

In [8]:
p.plotCircleHeatMap(plot_df, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width= 3000)

Loading BokehJS ...

# Step 4: Create a HeatMap with both +/- differences in median

 Read in the pos_neg_df.csv to create a df with only genes that have a pos and neg difference in median in different cancers. Slice out the genes that have a pos and neg difference in median in the pathway using the list of genes with a significant p-value in the pathway. 

In [9]:
pos_neg_df = pd.read_csv(root+R'\pos_neg_df.csv')

In [10]:
get = pos_neg_df.Proteomics.isin(genes) # bool df where True has both pos and neg
genes_pn = pos_neg_df[get] # Keep only genes with pos and neg
genes_pn.Proteomics.unique()

array(['USP39', 'HNRNPD', 'NUP37', 'YBX1', 'NUP153', 'DDX39A', 'CD2BP2',
       'HNRNPA1', 'FUS', 'HNRNPA3', 'NUP210'], dtype=object)

In [11]:
# Only include p-values < a certain cutoff
a = 0.01
genes_pn = genes_pn.loc[genes_pn['P_Value'] <= a]

In [12]:
p.plotCircleHeatMap(genes_pn, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width=600)

Loading BokehJS ...

# Sig in multiple cancers

In [13]:
mult_sig_df = pd.read_csv(root+R'\mult_sig_pval_heatmap.csv')

mult_sig_list = list(mult_sig_df.Proteomics.unique()) # list of genes with a sig pval in > 1 cancer
enr2 = gp.enrichr(gene_list = mult_sig_list, description='Tumor_partition', gene_sets='Reactome_2016', 
                       outdir='/Enrichr')

In [23]:
enr2.res2d.head(4)

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,Gene Expression Homo sapiens R-HSA-74160,74/1631,2.081067e-17,3.184033e-14,0,0,2.917745,112.073703,WDR3;WDR4;PTEN;HNRNPU;NAT10;ZC3H8;ADAR;PPP2R2A...,Reactome_2016
1,"Cell Cycle, Mitotic Homo sapiens R-HSA-69278",37/462,2.789117e-16,2.133675e-13,0,0,5.150262,184.459924,TOP2A;AHCTF1;PCNA;MCM7;DCTN1;BUB1B;PPP2R2A;ORC...,Reactome_2016
2,Cell Cycle Homo sapiens R-HSA-1640170,38/566,3.204908e-14,1.634503e-11,0,0,4.317544,134.152603,TOP2A;AHCTF1;PCNA;MCM7;DCTN1;BUB1B;PPP2R2A;ORC...,Reactome_2016
3,Processing of Capped Intron-Containing Pre-mRN...,23/193,3.858260e-14,1.475784e-11,0,0,7.663729,236.701740,RANBP2;DDX5;NUP210;CPSF1;NUP155;NCBP1;NUP133;D...,Reactome_2016


In [15]:
index2 = 3
trans2 = enr2.res2d.Genes[index2]
genes_mult = trans2.split(';')
print(enr2.res2d.Term[index2])
print('total genes:',len(genes_mult))

Processing of Capped Intron-Containing Pre-mRNA Homo sapiens R-HSA-72203
total genes: 23


In [16]:
bool_df = mult_sig_df.Proteomics.isin(genes_mult)
plot_df2 = mult_sig_df[bool_df]
plot_df2.Proteomics.unique()

array(['USP39', 'PRPF6', 'NCBP1', 'TPR', 'THOC2', 'DDX39A', 'POLR2B',
       'RANBP2', 'HNRNPH1', 'NUP155', 'PRPF3', 'THOC3', 'CPSF3', 'THOC7',
       'NUP85', 'DDX5', 'RAE1', 'ELAVL1', 'CPSF1', 'HNRNPU', 'DHX9',
       'NUP133', 'NUP210'], dtype=object)

In [17]:
# Only include p-values < a certain cutoff
a = 0.01
plot_df2 = plot_df2.loc[plot_df2['P_Value'] <= a]

In [19]:
p.plotCircleHeatMap(plot_df2, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=300, plot_width=800)

Loading BokehJS ...

In [20]:
get = pos_neg_df.Proteomics.isin(genes_mult) # bool df where True has both pos and neg
genes_pn_mult = pos_neg_df[get] # Keep only genes with pos and neg
genes_pn_mult.Proteomics.unique()

array(['USP39', 'DDX39A', 'NUP210'], dtype=object)

In [21]:
# Only include p-values < a certain cutoff
a = 0.01
genes_pn_mult = genes_pn_mult.loc[genes_pn_mult['P_Value'] <= a]

In [22]:
p.plotCircleHeatMap(genes_pn_mult, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width=400)

Loading BokehJS ...

# Check other pathways the genes (+/- and sig in mult. cancers) are part of

In [21]:
gnm = list(genes_pn_mult.Proteomics.unique())
print(gnm)
enr3 = gp.enrichr(gene_list = gnm, description='Tumor_partition', gene_sets='KEGG_2016', 
                       outdir='/Enrichr')

['USP39', 'DDX39A', 'NUP210']


2020-06-18 16:10:28,236 Warning: No enrich terms using library KEGG_2016 when cutoff = 0.05


In [22]:
enr3.res2d.head()

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,Spliceosome Homo sapiens hsa03040,1/134,0.019966,1.0,0,0,49.751244,194.711483,USP39,KEGG_2016
1,RNA transport Homo sapiens hsa03013,1/172,0.025580,1.0,0,0,38.759690,142.091094,NUP210,KEGG_2016


In [23]:
# Look at p-values 
certain_gene = 'PRPF6'
all_df = pd.read_csv(root+R'\all_heatmap.csv')
gene_df = all_df.loc[all_df['Proteomics'] == certain_gene]
gene_df

,Proteomics,P_Value,Medians,Cancer
212,PRPF6,0.000050,0.391870,Gbm
12022,PRPF6,0.006819,0.140160,Hnscc
27645,PRPF6,0.359535,0.144750,Luad
37501,PRPF6,0.393073,0.107700,Lscc
47320,PRPF6,0.161801,0.068850,Brca
58451,PRPF6,0.338623,-0.006257,Ov
65845,PRPF6,0.000959,-0.193600,En
82570,PRPF6,0.803451,-0.063000,Colon
